In [4]:
from Bio import SeqIO
import pandas as pd


# mbs_consensus_names: pre kazdy virus nazov(id)
mbs_consensus_names = {
                        'h1n1': 'run230320_UVZ_BA_23-vsp-1337_A__WISCONSIN__588__2019__H1N1_',
                        'h3n2': 'run230320_UVZ_BA_23-vsp-2129_A__DARWIN__6__2021__H3N2_',
                        'rsv_b': 'run230320_UVZ_BA_23-vsp-902_hRSV__B__Australia__VIC-RCH056__2019:EPI_ISL_1653999:2019-03-04', 
                        'sars_cov_2_122': 'Consensus_uvzsr-BA_24_00001-G12-BA_24_00000122_threshold_0.5_quality_20',  # mozno bude treba zmenit hlavicku v consenzus sekvencii pre sars_cov_2 a zjednotit s generovanymi vysledkami
                        'sars_cov_2_139': 'Consensus_uvzsr-BA_24_00002-A09-BA_24_00000139_threshold_0.5_quality_20',
                        'sars_cov_2_4964': 'run230710_UVZ_BA_23-vsp-4964_NC_045512.2',
                        'rsv_a': 'run230320_UVZ_BA_23-vsp-1658_hRSV__A__England__397__2017:EPI_ISL_412866:2017-01-01'
                       }

parameteres_df = pd.read_csv('./data/parameters_set.tsv', sep='\t')

mbs_consensus_path_dict = {
    'h1n1': './consensus_baseline/1337_consensus-h1n1_2019-wgs.fa',
    'h3n2': './consensus_baseline/2129_consensus-h3n2_2021-wgs.fa',
    'rsv_b': './consensus_baseline/902_consensus-rsv_b_2019-wgs.fa',
    'rsv_a': './consensus_baseline/1658_consensus-rsv_a_2017-wgs.fa',
    'sars_cov_2_139': './consensus_baseline/139_consensus-sars_cov_2-wgs.fa',
    'sars_cov_2_122': './consensus_baseline/122_consensus-sars_cov_2-wgs.fa',
    'sars_cov_2_4964': './consensus_baseline/4964_consensus-sars_cov_2-wgs-respiratory.fa'
}


In [2]:
virus = 'sars_cov_2'
consensus_baseline_path = mbs_consensus_path_dict[virus]

mbs_consensus = SeqIO.to_dict(SeqIO.parse(consensus_baseline_path, "fasta"))

mbs_consensus_seq = mbs_consensus[list(mbs_consensus.keys())[0]].seq

len(mbs_consensus_seq)

29831

In [5]:
mbs_consensus_path_dict.keys()

dict_keys(['h1n1', 'h3n2', 'rsv_b', 'rsv_a', 'sars_cov_2_139', 'sars_cov_2_122', 'sars_cov_2_4964'])

In [6]:
col_names = ['fragment', 'lenght', 'n_count']
all_virus_df = pd.DataFrame(columns=col_names)

for virus in mbs_consensus_path_dict.keys():

    print(virus)

    cons_dict = SeqIO.to_dict(SeqIO.parse(mbs_consensus_path_dict[virus], "fasta"))
    
    virus_lenght = 0
    virus_n_count = 0

    for fragment in cons_dict.keys():
        
        if virus == 'rsv_a' or virus == 'rsv_b' or virus == 'sars_cov_2_139' or virus == 'sars_cov_2_122' or virus == 'sars_cov_2_4964':
            fragment_name = virus
        else:
            fragment_name = virus + '_' + fragment.split('_')[-1]

        lenght_seq = len(cons_dict[fragment].seq)
        n_count = cons_dict[fragment].seq.count('N')

        virus_lenght += lenght_seq
        virus_n_count += n_count

        fragment_df = pd.DataFrame([[fragment_name, lenght_seq, n_count]], columns=col_names)
        all_virus_df = pd.concat([all_virus_df, fragment_df], ignore_index=True)
        # print(fragment_name, len(cons_dict[fragment].seq), cons_dict[fragment].seq.count('N'))

    virus_df = pd.DataFrame([[virus, virus_lenght, virus_n_count]], columns=col_names)
    all_virus_df = pd.concat([all_virus_df, virus_df], ignore_index=True)

all_virus_df


h1n1
h3n2
rsv_b
rsv_a
sars_cov_2_139
sars_cov_2_122
sars_cov_2_4964


fragment lenght n_count
0          h1n1_PB2   2316       0
1          h1n1_PB1   2316       0
2           h1n1_PA   2208       0
3           h1n1_HA   1752       0
4           h1n1_NP   1540       0
5           h1n1_NA   1433       0
6           h1n1_MP   1002       0
7           h1n1_NS    865       0
8              h1n1  13432       0
9          h3n2_PB2   2280       0
10         h3n2_PB1   2274       0
11          h3n2_PA   2151       0
12          h3n2_HA   1718       0
13          h3n2_NP   1497       0
14          h3n2_NA   1439       1
15          h3n2_MP    982       0
16          h3n2_NS    838       0
17             h3n2  13179       1
18            rsv_b  15224       3
19            rsv_b  15224       3
20            rsv_a  15221       0
21            rsv_a  15221       0
22   sars_cov_2_139  29835      15
23   sars_cov_2_139  29835      15
24   sars_cov_2_122  29835      22
25   sars_cov_2_122  29835      22
26  sars_cov_2_4964  29847       0
27  sars_cov_2_4964  29847       0

In [8]:
all_virus_df.to_csv('./consensus_baseline/virus_fragment_characteristics.tsv', sep='\t', index=False)

In [5]:
# count in
import gzip
from Bio import SeqIO

def count_read_lengths(file_path):
    with gzip.open(file_path, 'rt') as fastq_file:
        lengths = [len(record.seq) for record in SeqIO.parse(fastq_file, 'fastq')]
    return lengths

file_path = '/home/gaziova/Downloads/uvzsr-BA_24_00002-A09-BA_24_00000139/uvzsr-BA_24_00002-A09-BA_24_00000139_R1.fastq.gz'
read_lengths = count_read_lengths(file_path)


print("Read lengths:", read_lengths)
print("Average read length:", sum(read_lengths) / len(read_lengths))
